In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import sklearn
import numpy
import pickle
import torch.utils 
from torch.utils.data import Dataset,DataLoader
import torch.utils.data
from sklearn.preprocessing import StandardScaler


In [ ]:
class Auto_encoder_model (nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_fc=nn.Linear(input_dim,5000)
        self.hidden_fc=nn.Linear(5000,4000)
        self.hidden_fc2=nn.Linear(4000,3000)
        self.hidden_fc9=nn.Linear(3000,2000)
        self.hidden_fc10=nn.Linear(2000,1000)
        self.hidden_fc11=nn.Linear(1000,500)
        self.hidden_fc12=nn.Linear(500,200)
        self.hidden_fc13=nn.Linear(200,50)


        self.output_fc=nn.Linear(50,output_dim)
        self.dropout = nn.Dropout(0.2)
        self.dropout_mid = nn.Dropout(0.05)
        self.dropout_small=nn.Dropout(0.05)

    def forward(self,x):
       batch_size=x.shape[0]
       x = x.view(batch_size, -1)
       x1=nn.functional.relu(self.input_fc(x))
       x1=self.dropout_mid(x1)
       x1=nn.functional.relu(self.hidden_fc(x1))
       x1=self.dropout_mid(x1)
       x1=nn.functional.relu(self.hidden_fc2(x1))
       x1=self.dropout_mid(x1)
       x1=nn.functional.relu(self.hidden_fc9(x1))
       x1=self.dropout_mid(x1)
       x1=nn.functional.relu(self.hidden_fc10(x1))
       x1=self.dropout_mid(x1)

       x1=nn.functional.relu(self.hidden_fc11(x1))
       x1=self.dropout_small(x1)
       x1=nn.functional.relu(self.hidden_fc12(x1))
       x1=self.dropout_small(x1)
       x1=nn.functional.relu(self.hidden_fc13(x1))
       x1=self.dropout_small(x1)
       #output=nn.functional.sigmoid(self.output_fc(x1))
       output=(self.output_fc(x1))

       return output 




class train_dataset(Dataset):
    def __init__(self,x_feature,y_label):
        self.x=x_feature
        self.y=y_label


    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        label=[0,0]
        if self.y[index]==1:
            label=[1,0]
        else:
            label=[0,1]
        
        feature=self.x[index]
        feature_tensor=torch.tensor(feature).float()
        label_tensor=torch.tensor(label).float()

        return feature_tensor,label_tensor

def calculate_accuracy(y_pred, y):
  
    top_pred = y_pred.argmax(1)
    top_y= y.argmax(1)
    correct = top_pred.eq(top_y).sum()
    #print(top_pred)
    acc = correct.float() / y.shape[0]
    #print(correct)
    return acc

def model_train(model,dataloader,optimizer,citerion,device):
    epoch_loss=0
    epoch_acc=0
    model.train()

    for i,data in enumerate(dataloader):
        feature,label=data
        feature=feature.to(device)
        label=label.to(device)

        optimizer.zero_grad()
        y_pred=model(feature)
        loss=citerion(y_pred,label)
        acc=calculate_accuracy(y_pred,label)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
        epoch_acc+=acc.item()

    return epoch_loss/len(dataloader),epoch_acc/len(dataloader)


def model_evaluate(model,dataloader,criterion,device):
    epoch_loss=0
    epoch_acc=0

    model.eval()
    with torch.no_grad():
        for i,data in enumerate(dataloader):
            feature,label=data
            feature=feature.to(device)
            label=label.to(device)

            y_pred=model(feature)

            loss=criterion(y_pred,label)
            acc=calculate_accuracy(y_pred,label)

            epoch_loss+=loss.item()
            epoch_acc+=acc.item()
    return epoch_loss/len(dataloader),epoch_acc/len(dataloader)



In [3]:
dataset_path = "dataset_apoe4.pickle"

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(pd.__version__)
print(sklearn.__version__)


with open(dataset_path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


input_dim=X_train.shape[1]
print("input dimensions",input_dim)

scaler = StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)
#print(X_test)


model=Auto_encoder_model(input_dim,2) 


optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

training_data=train_dataset(X_train,y_train)
testing_data=train_dataset(X_test,y_test)

training_dataloader=DataLoader(training_data,batch_size=16,shuffle=True)
testing_dataloader=DataLoader(testing_data,batch_size=16,shuffle=True)
EPOCHS=200

for epoch in range(EPOCHS):


    train_loss,train_acc=model_train(model,training_dataloader,optimizer,criterion,device)

    test_loss,test_acc=model_evaluate(model,testing_dataloader,criterion,device)
    print(f'\t E {epoch} Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t E {epoch} Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}%')


True
NVIDIA A100-SXM4-80GB
2.2.2
1.3.2
X_train shape: (683, 10193)
y_train shape: (683,)
X_test shape: (151, 10193)
y_test shape: (151,)
input dimensions 10193
	 E 0 Train Loss: 0.668 | Train Acc: 59.98%
	 E 0 Test. Loss: 0.659 |  Test. Acc: 63.84%
	 E 1 Train Loss: 0.665 | Train Acc: 62.00%
	 E 1 Test. Loss: 0.654 |  Test. Acc: 63.84%
	 E 2 Train Loss: 0.670 | Train Acc: 61.73%
	 E 2 Test. Loss: 0.654 |  Test. Acc: 63.84%
	 E 3 Train Loss: 0.667 | Train Acc: 61.93%
	 E 3 Test. Loss: 0.652 |  Test. Acc: 64.64%
	 E 4 Train Loss: 0.669 | Train Acc: 61.60%
	 E 4 Test. Loss: 0.650 |  Test. Acc: 65.45%
	 E 5 Train Loss: 0.668 | Train Acc: 61.73%
	 E 5 Test. Loss: 0.658 |  Test. Acc: 63.84%
	 E 6 Train Loss: 0.667 | Train Acc: 61.67%
	 E 6 Test. Loss: 0.651 |  Test. Acc: 64.64%
	 E 7 Train Loss: 0.665 | Train Acc: 61.87%
	 E 7 Test. Loss: 0.652 |  Test. Acc: 64.64%
	 E 8 Train Loss: 0.665 | Train Acc: 62.00%
	 E 8 Test. Loss: 0.644 |  Test. Acc: 66.25%
	 E 9 Train Loss: 0.666 | Train Acc: 61

In [ ]:
# Test Acc: 64.64%